In [17]:
import numpy as np
import pandas as pd
import requests
import json
import urllib
import tensorflow as tf
import tensorflow_datasets
from tensorflow import keras
from gensim.models.callbacks import CallbackAny2Vec
%config Completer.use_jedi =False
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import pickle
from sklearn import preprocessing
import gensim
from platform import python_version
from tensorflow.keras.models import model_from_json

In [18]:
modified_embedding_array=np.load('picks_and_bans\\modified_embedding_array_30.npy')
modified_names_array=np.load('picks_and_bans\\modified_names_array.npy')
modified_pick_array=np.load('picks_and_bans\\modified_pick_array.npy')
modified_bans_array=np.load('picks_and_bans\\modified_bans_array.npy')

In [21]:
test_start=modified_embedding_array.shape[0]-int(0.05*modified_embedding_array.shape[0])
cv_start=test_start-int(0.1*modified_embedding_array.shape[0])
test_picks=modified_embedding_array[test_start:]
testlabel_picks=modified_pick_array[test_start:]
cv_picks=modified_embedding_array[cv_start:test_start]
cvlabel_picks=modified_pick_array[cv_start:test_start]
train_picks=modified_embedding_array[:cv_start]
tlabel_picks=modified_pick_array[:cv_start]
tlabel_picks=tlabel_picks.reshape((tlabel_picks.shape[0],tlabel_picks.shape[1],1))
one_labels=tf.one_hot(tlabel_picks,depth=120)
one_labels=tf.reshape(one_labels,(tlabel_picks.shape[0],tlabel_picks.shape[1],120))
one_cvlabels=tf.one_hot(cvlabel_picks,depth=120)
one_cvlabels=tf.reshape(one_cvlabels,(cvlabel_picks.shape[0],cvlabel_picks.shape[1],120))
one_testlabels=tf.one_hot(testlabel_picks,depth=120)
one_testlabels=tf.reshape(one_testlabels,(testlabel_picks.shape[0],testlabel_picks.shape[1],120))
hero_df=pd.read_csv('hero_df.csv')
hero_dict={}
for i in hero_df['sid']:
    hero_dict[i]=hero_df[hero_df['sid']==i]['localized_name'].values[0]
embed_df=pd.read_csv('modified_hero_to_embedding.csv')
embed_df.set_index('Heroes',inplace=True)

In [22]:
cv_start

89385

In [23]:
def pipe(arr,labels):

    data=tf.data.Dataset.from_tensor_slices(arr)
    vals=tf.data.Dataset.from_tensor_slices(labels)
    combined=tf.data.Dataset.zip((data,vals))
    finale=combined.map(lambda x,y: (x[:-1],y[-1]))
    return finale
def preds(arr):
    if len(arr.shape)!=2:
        data=tf.data.Dataset.from_tensor_slices(arr)
        data=data.map(lambda x: x[:-1])
        data=data.batch(1,drop_remainder=True)
    else:
        data=np.reshape(arr,(1,arr.shape[0],arr.shape[1]))
        data=tf.data.Dataset.from_tensor_slices(data)
        data=data.map(lambda x: x[:-1])
        data=data.batch(1,drop_remainder=True)
    return data

In [24]:
train=pipe(train_picks,one_labels)
train=train.batch(32,drop_remainder=True)
cv=pipe(cv_picks,one_cvlabels)
cv=cv.batch(1,drop_remainder=True)
test=pipe(test_picks,one_testlabels)
test=test.batch(1,drop_remainder=True)

In [27]:
model=keras.models.Sequential([
    keras.layers.GRU(128,recurrent_dropout=0.25,return_sequences=True,input_shape=[None,30]),
    keras.layers.GRU(128,recurrent_dropout=0.1),
    keras.layers.Dense(120,activation='softmax'),
    keras.layers.Lambda(lambda x: x/0.4)
])
model.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer="Nadam",metrics=[keras.metrics.TopKCategoricalAccuracy(3)])
history=model.fit(train,epochs=3,validation_data=cv)
model.save('Models\\lstm128_30topk.h5')

Epoch 1/3
2793/2793 [==============================] - 40s 14ms/step - loss: 4.1892 - top_k_categorical_accuracy: 0.1311 - val_loss: 4.0056 - val_top_k_categorical_accuracy: 0.1697
Epoch 2/3
2793/2793 [==============================] - 37s 13ms/step - loss: 3.9833 - top_k_categorical_accuracy: 0.1770 - val_loss: 3.9587 - val_top_k_categorical_accuracy: 0.1874
Epoch 3/3
2793/2793 [==============================] - 38s 14ms/step - loss: 3.8995 - top_k_categorical_accuracy: 0.1986 - val_loss: 3.9435 - val_top_k_categorical_accuracy: 0.1873


In [ ]:
model=keras.models.load_model('Models\\lstm128_30topk.h5')

In [28]:
#Taking the next best suggestion if any of the suggestions is a part of the bans list for that match
y_pred=model.predict(test)
y_true=[]
for i in test:
    y_true.append(i[1])
y_true=np.array(y_true)
y_true=y_true.reshape(y_true.shape[0]*y_true.shape[1],120)
inbans=0
modified_pred=np.copy(y_pred)
for i,j in enumerate(modified_pred):
    preds=np.argsort(-j)[:3]
    while np.intersect1d(preds,modified_bans_array[i+test_start]).shape[0]!=0:
        intersection=np.intersect1d(preds,modified_bans_array[i+test_start])
        modified_pred[i,intersection]=-10.0
        preds=np.argsort(-modified_pred[i])[:3]

In [29]:
topk=keras.metrics.TopKCategoricalAccuracy(k=3)
topk.update_state(y_true,modified_pred)
topk.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.2587027>